# Imports

In [1]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance
from scipy.spatial.distance import cdist
from shapely.geometry import LineString, MultiLineString, Polygon, MultiPolygon, LinearRing

# Input

In [2]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [3]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [4]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [5]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [6]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[1.22157599 1.22228672 1.22157599 ... 1.13724611 1.6508054  1.13724611]
[0.94448555 0.94503506 0.94448555 ... 0.87928424 1.27635273 0.87928424]
[0.74821541 0.74833477 0.74821541 ... 0.7337888  0.81333141 0.7337888 ]
(5999,)


### Multinomial Sampling

In [7]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[1 0 2 ... 0 0 0]


In [8]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(graph)[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[88.630486 ,  3.5589852, 91.10313  ],
       [90.12466  ,  4.1574683, 87.93393  ],
       [75.70441  ,  6.9390492, 84.51398  ],
       [90.45635  ,  4.1574683, 87.13314  ],
       [86.91348  ,  3.5589852, 93.11348  ]], dtype=float32)

# KNN extended graph

In [9]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [10]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for poly in indices:
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11764


# Edge Predictor

In [11]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [12]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [13]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [14]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [15]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 249)	1
  (0, 250)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  :	:
  (1498, 1446)	1
  (1498, 1447)	1
  (1498, 1450)	1
  (1498, 1451)	1
  (1498, 1453)	1
  (1498, 1490)	1
  (1498, 1492)	1
  (1498, 1494)	1
  (1498, 1495)	1
  (1498, 1496)	1
  (1498, 1497)	1
  (1499, 1201)	1
  (1499, 1202)	1
  (1499, 1205)	1
  (1499, 1206)	1
  (1499, 1207)	1
  (1499, 1208)	1
  (1499, 1257)	1
  (1499, 1258)	1
  (1499, 1265)	1
  (1499, 1489)	1
  (1499, 1490)	1
  (1499, 1491)	1
  (1499, 1493)	1
  (1499, 1497)	1
[[0.06218101 0.06217998 0.06217998 ... 0.06163289 0.06159952 0.06173376]
 [1.18470407 0.05970745 0.05970745 ... 0.06096624 0.06103535 0.06075266]
 [1.18477581 0.58602112 0.0595572  ... 0.06058806 0.06064415 0.06041435]
 ...
 [1.88740486 1.90758249 1.91994679 ... 0.06255528 0.06239804 

In [16]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [17]:
print(A_s)  # symmétrique
print(A_s.shape)

[[4.35579065e-23 4.33157335e-23 4.33581497e-23 ... 9.79439598e-21
  5.99966509e-21 7.17399893e-21]
 [4.33157335e-23 4.31251248e-23 4.34371052e-23 ... 1.04303516e-20
  6.34431728e-21 7.53983526e-21]
 [4.33581497e-23 4.34371052e-23 4.37505446e-23 ... 1.02820187e-20
  6.26151216e-21 7.44690542e-21]
 ...
 [9.79439598e-21 1.04303516e-20 1.02820187e-20 ... 1.21731570e-17
  6.91291112e-18 7.47841147e-18]
 [5.99966509e-21 6.34431728e-21 6.26151216e-21 ... 6.91291112e-18
  6.07665355e-18 4.49333851e-18]
 [7.17399893e-21 7.53983526e-21 7.44690542e-21 ... 7.47841147e-18
  4.49333851e-18 4.89021958e-18]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [18]:
igraph_g = igraphGraph(directed=extended_graph.is_directed()).from_networkx(extended_graph)
print(igraph_g.summary())

IGRAPH U--- 1500 11764 -- 
+ attr: _nx_name (v)


In [19]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[ 863  864 1294]
 [1466 1467 1468]
 [ 661  662  664]]
32574


In [20]:
triangles = np.array(igraph_g.vs['_nx_name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[ 75.93793     4.4479885  81.95952  ]
  [ 75.85339     4.4479885  82.0534   ]
  [ 75.85237     5.0095077  82.51254  ]]

 [[ 76.07001    -5.5423326  82.847664 ]
  [ 76.15477    -5.5423326  82.789406 ]
  [ 76.23637    -5.5423326  82.72679  ]]

 [[ 51.         -7.234973  -10.009508 ]
  [ 51.         -5.4119864 -11.939049 ]
  [ 51.         -4.303499  -12.67593  ]]]
32574


In [21]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(triangles))
p_init = (A_s_ij + A_s_ik + A_s_jk) / 3

print(p_init.shape)
print(p_init)

(32574,)
[9.37965871e-21 1.25460741e-19 5.01768731e-07 ... 3.86950923e-12
 5.02163751e-12 4.54471065e-12]


#### Calculate barycenter

In [22]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(32574, 3)
[[ 75.88123     4.635162   82.17515  ]
 [ 76.15372    -5.542333   82.78796  ]
 [ 51.         -5.6501527 -11.541496 ]]


#### KNN Tri

In [23]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [24]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(32574, 19, 3)


#### calculate e norm matrix

In [25]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(32574, 3)
[[0.12633029 0.79275435 0.72534025]
 [0.10285278 0.2056343  0.10285201]]


#### Calculate r

In [26]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(32574,)

In [27]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(32574, 19)
(32574, 19)


In [28]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(32574, 19, 5)


#### Calculate f

In [29]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

1.000000000000003
(32574,)
[3.06937787e-05 3.06937787e-05 3.06937941e-05 ... 3.06937787e-05
 3.06937787e-05 3.06937787e-05]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_1440\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [30]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[0])

(500, 3, 3)
[[51.         8.492921  87.695595 ]
 [51.        -1.8187612 81.39     ]
 [39.         7.4193583 85.26786  ]]


In [31]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 50
Number of edges: 234


In [32]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [33]:
keys_input = np.array(graph)
print("Les points avant K-NN : ", keys_input)
print("dim points avant K-NN : ", len(keys_input))  #return 5999

Les points avant K-NN :  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]
dim points avant K-NN :  5999


Récupération des points de l'espace de départ Ps

In [34]:
keys_output = np.array(extended_graph)
print("sampled points : ", keys_output)
print("dim sample points : ", len(keys_output))  #return: 50

sampled points :  [[88.630486   3.5589852 91.10313  ]
 [88.8296     2.9401166 91.24779  ]
 [88.28478    2.9401166 91.9578   ]
 ...
 [83.630974   8.114116  -4.8309755]
 [81.69544    7.5075808 -7.796847 ]
 [86.931366   7.5075808 -2.2793348]]
dim sample points :  1500


Distance Matrix

In [35]:
def matrix_distance_euclidienne_barycentres(barycentre_b, barycentre_b_hat):
    """
    Calculate distances between barycenters b et b_hat after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """
    return cdist(barycentre_b, barycentre_b_hat, metric='sqeuclidean').T

distances_matrix = matrix_distance_euclidienne_barycentres(keys_input, keys_output)
print("Squared Euclidean Distances Matrix Shape:", distances_matrix.shape)
print("Distances Matrix:\n", distances_matrix)




Squared Euclidean Distances Matrix Shape: (1500, 5999)
Distances Matrix:
 [[ 7824.78626084  5737.69896997  7820.29242132 ... 11183.8326739
  12106.13047905 11058.99882622]
 [ 7850.67764431  5760.06816828  7847.01759486 ... 11212.62209901
  12139.35426034 11087.44392098]
 [ 7960.6152675   5851.67541597  7956.92775949 ... 11311.69099855
  12224.41873126 11185.58395343]
 ...
 [  225.6353133    612.02396003   214.82632067 ...  1347.89921363
   2276.85473658  1349.711365  ]
 [  245.77877622   708.8458426    235.67957122 ...  1201.49370997
   2087.72454808  1207.03393528]
 [  245.51488909   566.34550151   235.67957349 ...  1571.79075703
   2576.46555168  1570.11276847]]


In [36]:
# Calculer le min des x de distances_matrix
def min_pour_x_list(distances_matrix, x_in):
    return np.min(distances_matrix[:, x_in])

# Calculer le min des y de distances_matrix
def min_pour_y_list(distances_matrix, y_out):
    return np.min(distances_matrix[y_out, :])


In [37]:
def d_P_Ps(keys_x, keys_y, p_y):
    """
    Calculate the Probabilistic Chamfer distance between two sets of points.

    Parameters:
    - keys_x (list): The input vertex set, a list of tuples.
    - keys_y (list): The sampled points, a list of tuples.
    - p_y (list): Their respective probabilities, a weight associated with each point in keys_y.

    Returns:
        chamfer_distance (float): The Probabilistic Chamfer distance between the two sets of points.
    """
    min_for_x = [p_y[i] * min_pour_x_list(distances_matrix, i) for i in range(len(keys_x))]
    min_for_y = [p_y[j] * min_pour_y_list(distances_matrix, j) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(min_for_y)
    sum_for_x = np.sum(min_for_x)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance


In [38]:
print("PROBA : ", normalized_inclusion_score)

PROBA :  [0.00022071 0.00022075 0.00022071 ... 0.00021646 0.00023992 0.00021646]


In [39]:
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Chamfer Distance:", prob_chamfer_dist)

Chamfer Distance: 4.538946063021137


## Probabilistic Surfaces Distance

barycentres b et b_hat

In [40]:
# Creation igraph
igraph_g_original = igraphGraph(directed=extended_graph.is_directed()).from_networkx(graph)
print(igraph_g_original.summary())

#Find triangles
triangles_ids_igraph_original = np.array(igraph_g_original.cliques(min=3, max=3))
triangles = np.array(igraph_g_original.vs['_nx_name'])[triangles_ids_igraph_original]
print("triangles shape : ", triangles.shape)
print(triangles[0])

# Calculate barycenters
b = np.mean(triangles, axis=1)
print("shape b : ", b.shape)
print("Barycentre b: ", b)

IGRAPH U--- 5999 17991 -- 
+ attr: _nx_name (v), index_triangle (v)
triangles shape :  (11994, 3, 3)
[[39.        8.793697 -4.666995]
 [51.        8.793697 -4.666995]
 [39.        8.793697 -5.333005]]
shape b :  (11994, 3)
Barycentre b:  [[43.          8.793697   -4.8889985 ]
 [55.          1.0515273   3.7313068 ]
 [55.         -8.515905   -2.804371  ]
 ...
 [75.05613    -0.94137865 21.        ]
 [75.01313    -0.3882971   8.099999  ]
 [75.03516    -0.72036856 25.300001  ]]


In [41]:
# Get finals selected triangles & calculate barycenters
b_hat = np.mean(selected_triangles, axis=1)
print(b_hat.shape)
print("B_HAT : ", b_hat)

(500, 3)
B_HAT :  [[47.          4.6978393  84.784485  ]
 [80.31373    -3.4051468  64.        ]
 [39.         -3.5383415   2.003548  ]
 ...
 [87.31728    -7.6234546  59.7       ]
 [82.77744    -5.8180504  59.7       ]
 [39.          0.11193975 86.07437   ]]


test matrice des normes euclidiennes entre b et b_hat

In [42]:
distances = matrix_distance_euclidienne_barycentres(b, b_hat)
print(distances.shape)
print("Distances euclidiennes entre les barycentres b et b_hat :\n", distances)


(500, 11994)
Distances euclidiennes entre les barycentres b et b_hat :
 [[8074.10967357 6646.91326676 7910.41073963 ... 4887.40768797
  6691.11455262 4353.73076529]
 [6286.82023725 4292.9621672  5129.72869552 ... 1882.71249962
  3162.00778693 1532.76126903]
 [ 215.58637901  280.05204622  303.89222732 ... 1667.65387959
  1344.03504688 1849.198191  ]
 ...
 [6405.28333268 4252.15683593 4951.99979399 ... 1692.67609932
  2866.29983319 1381.86322316]
 [5967.48696517 3951.27213778 4685.66126172 ... 1581.09066803
  2752.32691079 1269.28933677]
 [8365.70756894 7037.26310361 8229.87057224 ... 5535.827756
  7377.19845763 4992.74929551]]


In [43]:
minimums_b = np.min(distances, axis=1)
print(minimums_b.shape)
print(minimums_b[:5])

(500,)
[ 3.15451224 15.37948972  0.53263409 15.09775702  0.06572607]


In [44]:
d_f_S_Ss = np.sum(selected_triangles_indexes * minimums_b)
print(d_f_S_Ss)

85941036.95043153


## Triangle Collision Loss

In [45]:
def compute_lc_le_lo(p_t, m_c_e_o, Fs):
    """
    Compute the collision loss term L_c.

    Parameters:
    - p_t: 1D numpy array containing the probabilities of each triangle (indices)
    - m_c_t: 2D numpy array containing the number of faces penetrated by each triangle
    - Fs: 3D numpy array representing the vertices of triangles

    Returns:
    - L_c: Collision loss term
    """
    assert len(p_t) == len(m_c_e_o), "Input arrays must have the same length"

    penalty_per_triangle = p_t * m_c_e_o

    # Sum the penalties for all selected triangles
    total_penalty = np.sum(penalty_per_triangle)

    # Compute the collision loss term L_c
    L_c_e_o = (1 / len(Fs)) * total_penalty

    return L_c_e_o

# Example usage:
# Replace the arrays below with your actual data
# p_t = selected_triangles_indexes
# m_c_t = numpy array containing the number of faces penetrated by each triangle
# Fs = 3D numpy array representing the vertices of triangles
p_t = selected_triangles_indexes
Fs = triangles  # Given data

In [46]:
number_neigh_selected_barycenters = min(50, len(b_hat))
_, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=number_neigh_selected_barycenters).fit(b_hat).kneighbors(b_hat)

## Lc

In [47]:
mc = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    polygons_others_tri = MultiPolygon([Polygon(others_triangle) for others_triangle in others_triangles]).buffer(0)    # buffer 0 to correct invalid polygons => take the exterior of the shape


    intersection = lines_current_triangle.intersection(polygons_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        mc[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        mc[index_neigh_barycenters[0]] += 1
L_c = compute_lc_le_lo(p_t, mc, Fs)
print("L_c : ", L_c)

L_c :  2578.897948974487


## Le

In [48]:
me = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    lines_others_tri = MultiLineString([LineString([other_tri[0], other_tri[1], other_tri[2], other_tri[0]]) for other_tri in others_triangles])

    intersection = lines_current_triangle.intersection(lines_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        me[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        me[index_neigh_barycenters[0]] += 1
L_e = compute_lc_le_lo(p_t, me, Fs)
print("L_e : ", L_e)

L_e :  12267.803151575788


## Lo

In [ ]:
igraph_g = igraphGraph(directed=extended_graph.is_directed()).from_networkx(extended_graph)
print(igraph_g.summary())

triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(" IDS TRI", triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))


triangles = np.array(igraph_g.vs['_nx_name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print("TRI",triangles)
print(len(triangles))

In [ ]:
import numpy as np

def sample_points_from_triangle(triangle, num_points=100):
    v1, v2, v3 = triangle
    bary_coords = np.random.rand(num_points, 2)
    sqrt_bary_coords = np.sqrt(bary_coords[:, 0])

    u = sqrt_bary_coords
    v = bary_coords[:, 1]

    """
    La formule spécifique est dérivée de l'expression générale d'interpolation barycentrique 
    sommets A, B et C
    coord barycentriques: u et v
    coord cartésiennes: x,y,z 
    """
    x_coords = (1 - u - v) * v1[0] + u * v2[0] + v * v3[0]
    y_coords = (1 - u - v) * v1[1] + u * v2[1] + v * v3[1]
    z_coords = (1 - u - v) * v1[2] + u * v2[2] + v * v3[2]

    sampled_points = np.column_stack((x_coords, y_coords, z_coords))
    return sampled_points


""""

Calcul des distances : Ces points échantillonnés sont utilisés pour calculer leurs distances par rapport aux 50 triangles les plus proches.
Objectif d'appartenance : L'objectif est d'attribuer chaque point échantillonné à un seul triangle. 
Pour déterminer cette appartenance, nous mesurons les aires A1, A2, A3 en remplaçant chaque sommet du triangle par le point de requête. 
Ensuite, nous vérifions si la somme de ces trois aires est égale à l'aire du triangle.
Prise en compte des plans parallèles : Pour les triangles partageant des plans parallèles, 
nous ajustons légèrement la tolérance de distance sur l'axe vertical par rapport au plan du triangle.
Application de la pénalité : Enfin, une pénalité est appliquée à tous les triangles partageant un point échantillonné. 
Cette pénalité, similaire à la perte de croisement d'arêtes (Edge Crossing), est proportionnelle au nombre de triangles qui se chevauchent.

"""
# Prélèvement de 100 points de chaque triangle
sampled_points_list = [sample_points_from_triangle(triangle, num_points=100) for triangle in triangles]

# Affichage des points échantillonnés pour le premier triangle (vous pouvez boucler pour afficher pour tous les triangles)
print("Points échantillonnés pour le premier triangle :")
print(sampled_points_list[0])

# Convertir les triangles en une liste plate pour faciliter la recherche
flat_triangles = np.vstack(triangles)

sampled_points_flat = np.vstack(sampled_points_list)

# Sélection des coordonnées x, y, et z des triangles
flat_triangles_xyz = flat_triangles[:, :3]

#modèle k-NN
knn_model = NearestNeighbors(n_neighbors=50, algorithm='auto').fit(flat_triangles_xyz)

# Recherche des indices des triangles les plus proches pour chaque point échantillonné
distances, indices = knn_model.kneighbors(sampled_points_flat)

# Initialisation des variables pour stocker les résultats
assignment_results = []

# Calcul des distances par rapport aux 50 triangles les plus proches
for i, point in enumerate(sampled_points_list):
    closest_triangle_indices = indices[i]
    closest_triangles = flat_triangles[closest_triangle_indices]

    # Calcul des aires et détermination de l'appartenance au triangle
    for triangle in closest_triangles:
        # Remplacer chaque sommet du triangle par le point de requête
        modified_triangle = np.copy(triangle)
        modified_triangle[0] = point

        # Calcul des aires A1, A2, A3
        area_original = calculate_triangle_area(triangle)
        area_modified = calculate_triangle_area(modified_triangle)

        # Vérification si la somme des aires est égale à l'aire du triangle
        if np.isclose(area_original, area_modified, rtol=1e-5):
            assignment_results.append((point, triangle))
            break  # On attribue le point au premier triangle vérifié

# Prise en compte des plans parallèles et ajustement de la tolérance (à implémenter)

# Application de la pénalité pour les triangles se chevauchant (à implémenter)

# Affichage des résultats (à remplacer par le traitement final souhaité)
for point, triangle in assignment_results:
    print(f"Point {point} attribué au triangle {triangle}")


In [49]:
mo = np.zeros((500))
